In [1]:
import os

In [2]:
%pwd

'd:\\github\\Github\\Wine-Quality-End-to-End-ML-Project\\notebook'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\github\\Github\\Wine-Quality-End-to-End-ML-Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    scaler_name: str
    max_depth: int
    n_estimators: int
    random_state: int
    target_column: str

In [6]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.RandomForestClassifier
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            scaler_name = config.scaler_name, 
            max_depth = params.max_depth,
            n_estimators = params.n_estimators,
            random_state = params.random_state,            
            target_column = schema.name
            
        )

        return model_trainer_config

In [8]:
import pandas as pd
import os
from mlProject import logger
# from sklearn.linear_model import ElasticNet
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import joblib

In [ ]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        X_train = train_data.drop([self.config.target_column], axis=1)
        X_test = test_data.drop([self.config.target_column], axis=1)
        y_train = train_data[[self.config.target_column]].squeeze()
        y_test = test_data[[self.config.target_column]].squeeze()

        # scaler=MinMaxScaler()
        scaler=StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
        
        joblib.dump(scaler, os.path.join(self.config.root_dir, self.config.scaler_name))

        rfc=RandomForestClassifier(max_depth=self.config.max_depth, n_estimators=self.config.n_estimators, random_state=self.config.random_state)
        rfc.fit(X_train, y_train)
        
        # lr = ElasticNet(alpha=self.config.alpha, l1_ratio=self.config.l1_ratio, random_state=42)
        # lr.fit(train_x, train_y)

        joblib.dump(rfc, os.path.join(self.config.root_dir, self.config.model_name))


In [10]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2025-02-16 14:28:39,383: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-02-16 14:28:39,386: INFO: common: yaml file: params.yaml loaded successfully]
[2025-02-16 14:28:39,390: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-02-16 14:28:39,392: INFO: common: created directory at: artifacts]
[2025-02-16 14:28:39,393: INFO: common: created directory at: artifacts/model_trainer]
